# Lecture 17 Dimension Reduction
This notebook will cover how high dimensional data from intracellular metabolite concentrations can be reduced to lower dimensions using PCA, LDA, and t-SNE.

### Set up imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.manifold import TSNE

### Load data

In [ ]:
# load the rhodococcus intracellar metabolite dataset


### Preprocess the data using the standard scaler

In [ ]:
# isolate the sample labels


# drop the sample column


In [ ]:
# standardize the data using the StandardScaler


### Run PCA

In [ ]:
# define and run PCA


In [ ]:
# convert the PCA results to a dataframe



In [ ]:
# add the sample labels back to the dataframe


In [ ]:
# find the explained variance of the axes


In [ ]:
# plot the PCA results


In [ ]:
# Make a dataframe of the loading scores


In [ ]:
# plot a bar chart of the metabolites with the top 10 loading scores


### Run LDA

In [ ]:
# write a function to convert a label to group name


In [ ]:
# use list comprehension to convert the labels to group names


In [ ]:
# define and run LDA


In [ ]:
# convert the LDA results to a dataframe


In [ ]:
# plot the LDA results


### Run t-SNE

In [ ]:
# define and run t-SNE


In [ ]:
# create a dataframe from the t-SNE output


In [ ]:
# plot the t-SNE results
